In [ ]:
import os

In [ ]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8' #name of our pre-trained model
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

In [ ]:
paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'export'), 
    'TFJS_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH':os.path.join('Tensorflow','protoc')
 }

In [ ]:
files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [ ]:
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

In [ ]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'], 'ckpt-3')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])

In [ ]:
import numpy as np
import cv2
import pickle
import tensorflow as tf
import time
%matplotlib inline
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
import math
import statistics as stat

In [ ]:
caspath = 'C:/Users/850/anaconda3/envs/tfod/Lib/site-packages/cv2/data/haarcascade_frontalface_alt2.xml'
face_cascade = cv2.CascadeClassifier(caspath)
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read("trainner.yml")
labels = {}
with open("labels.pickle", 'rb') as f:
    og_lables = pickle.load(f)
    labels = {v:k for k,v in og_lables.items()} # to invert lables 

cap = cv2.VideoCapture(0)
stor_conf = []
while True:
    key = cv2.waitKey(20) & 0xFF
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor = 1.5,minNeighbors = 5 )
    for (x, y, w, h) in faces:
        #print (x, y , w, h)
        roi_gray = gray[y:y+h, x:x+w] # roi is region of interst
        roi_color = frame[y:y+h, x:x+w]
        canny = cv2.Canny(frame,100,100)
        id_, conf = recognizer.predict(roi_gray)
        
        
     
            
        if conf >= 60 and id_ == 0:
            print(id_)           
            print(labels[id_], conf)
            stor_conf.append(conf)
            
            font = cv2.FONT_HERSHEY_SIMPLEX
            name = labels[id_]
            color = (255,255,255)
            stroke = 2 
            
            cv2.putText(frame, name, (x,y), font, 1, color, stroke, cv2.LINE_AA)
        if id_ != 0 or conf < 60:
            font = cv2.FONT_HERSHEY_SIMPLEX
            color = (255,255,255)
            stroke = 2 
            cv2.putText(frame, "Unknown", (x,y), font, 1, color, stroke, cv2.LINE_AA)
        
            
            
        color = (255,0,0)
        stroke = 2
        end_cord_x = x+w #width
        end_cord_y = y+h #height
        cv2.rectangle(frame, (x, y), (end_cord_x, end_cord_y), color, stroke)
    
    cv2.imshow('frame', frame)
   
    if len(stor_conf) > 75 and stat.mean(stor_conf) > 70:
        cap.release()
        cv2.destroyAllWindows()
        devices = AudioUtilities.GetSpeakers()
        interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
        volume = cast(interface, POINTER(IAudioEndpointVolume))


        cap1 = cv2.VideoCapture(0)
        width = int(cap1.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap1.get(cv2.CAP_PROP_FRAME_HEIGHT))

        n = 0 # We will need p and n this for the audio control
        p = 0

        while cap1.isOpened(): 
            ret, frame = cap1.read()
            image_np = np.array(frame)

            input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
            detections = detect_fn(input_tensor)

            num_detections = int(detections.pop('num_detections'))
            detections = {key: value[0, :num_detections].numpy()
                          for key, value in detections.items()}
            detections['num_detections'] = num_detections

            # detection_classes should be ints.
            detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

            label_id_offset = 1 # we have labeled our hand detection from 1 so we cn't have it start from 0.
            image_np_with_detections = image_np.copy()
            
            #defining boxes and labels when activating our webcam.
            viz_utils.visualize_boxes_and_labels_on_image_array(
                        image_np_with_detections,
                        detections['detection_boxes'],
                        detections['detection_classes']+label_id_offset,
                        detections['detection_scores'],
                        category_index,
                        use_normalized_coordinates=True,
                        max_boxes_to_draw=5,
                        min_score_thresh=.8,
                        agnostic_mode=False)

            theLabels = detections['detection_classes']+label_id_offset
            labelsList = theLabels.tolist()

            #   This next part is for the audio change. We assign one hand label for volume up and another one
            # for volume down.
            if labelsList[0] == 1:
                n += 5
            if labelsList[0] == 2:
                p += 5
            print(detections['detection_classes']+label_id_offset)
            if n >= 30:
                currentVolumeDb = volume.GetMasterVolumeLevel()
                volume.SetMasterVolumeLevel(currentVolumeDb + 0.5, None) # 0.5 represents the volume change when recognising the label
                print(currentVolumeDb) 
                n = 0

            if p >= 30:
                currentVolumeDb = volume.GetMasterVolumeLevel()
                volume.SetMasterVolumeLevel(currentVolumeDb - 0.5, None)
                print(currentVolumeDb)
                p = 0

            cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))

            if cv2.waitKey(10) & 0xFF == ord('q'):
                cap1.release()
                cv2.destroyAllWindows()
                break
            if  key == ord('q'):
                break
    
     
cap.release()
cv2.destroyAllWindows()